In [107]:
import numpy as np
from numpy import genfromtxt as csv_load
import sys
import csv

def checkTOs(v):
    tos = []
    count = 0
    for s in range(0,len(v)-1):
        count += 1
        if (v[s] == 0 and v[s+1] == 1):
            tos.append(count)
    return tos

def checkHSs(v):
    hss = []
    count = 0
    for s in range(0,len(v)-1):
        count += 1
        if (v[s] == 1 and v[s+1] == 0):
            hss.append(count)
    return hss

def evaluate(ref_tr, pred_tr, tolerance):
    positives = len(ref_tos)
    tp = 0;
    fp = 0;
    fn =0;
    diffs = []
    for p_tr in pred_tr:
        found = False
        for r_tr in ref_tr:
            diff = abs(r_tr-p_tr)
            if (diff <= tolerance):
                #print("r_tr:" + str(r_tr) + " p_tr:" + str(p_tr) + " diff:" + str(diff))
                tp += 1
                diffs.append(diff)
                found = True
                break
        if not found:        
            fp += 1            
    fn = positives - tp
    precision = tp/(tp+fp)
    recall = tp/positives
    f1 = 2*(precision*recall)/(precision+recall)
    ME = np.mean(diffs)
    variance = np.var(diffs)
    SD = np.std(diffs)
    
    return positives, tp, fp, fn, precision, recall, f1, diffs, ME, variance, SD

## -- This is required to match the transition arrays when they differ in dimension 
def delete_false_transitions(x, y, thr):
    limit = min(x.shape[0], y.shape[0])
    x_list = []    # It will contain the false transitions in the x array
    y_list = []    # It will contain the false transitions in the y array
    for i in range(0,limit-1):
        diff = abs(x[i] - y[i])    # Compare two transitions with the same index in the two arrays
        if diff > thr:    # If they differ more than the threshold:
            if abs(x[i+1] - y[i]) < thr:    # Compare the next x transition with the current y -> If < thr:
                x_list.append(i)     # Fill x_list with the transition i, since it's a false transition
                x = np.delete(x,x_list)    # Delete from the x vector the i-th transition
                break    # Leave the cycle
            if abs(x[i] - y[i+1]) < thr:    # Compare the current x transition with the nexy y -> If < thr
                y_list.append(i)    # Fill y_list with the i-th transition
                y = np.delete(y, y_list)    # Delete from the y vector the i-th transition
                break
    
    # It works one one transition per time, thus, to cycle it
    # if the two arrays have different dimensions AND at least a transition has been found
    # in the previous run of the function --> recall the function with the new, "clean" arrays.
    if x.shape[0] != y.shape[0] and (x_list != [] or y_list != []):
        delete_false_transitions(x, y, thr)
    return x, y


################################ --- Data --- ################################ 
model = 'FF4'
fold = 1

subjects = [20, 25, 21, 6, 4]
in_path_log = '../subjects/min-max/windows_20/tr-False_sliding_1_c-False/folds_inter/Report_' + model + '/Fold_' + str(fold) + '/'
log = in_path_log + 'fold_' + str(fold) + '_log.txt'
old_stdout = sys.stdout
log_file = open(log,"w")


sys.stdout = log_file
print("MODEL " + str(model) + "  ----  FOLD " + str(fold) + "\n\n\n")
for subj in subjects:
    in_predicted = '../subjects/min-max/windows_20/tr-False_sliding_1_c-False/folds_inter/Report_' + model + '/Fold_' + str(fold) + '/'
    file_predicted = in_predicted + 's' + str(subj) +  '_predicted_clean_350.csv'
    in_real = '../subjects/min-max/clean/baso_real/'
    file_real = in_real + 's' + str(subj) + '_baso_real.csv'


## -- Load data
    real = csv_load(file_real, delimiter = ',')
    predicted = csv_load(file_predicted, delimiter = ',')

## -- Check transitions, both Toe-Offs and Heel-Strikes
    ref_tos = checkTOs(real)
    ref_tos = np.asarray(ref_tos, dtype = int)
    ref_hss = checkHSs(real)
    ref_hss = np.asarray(ref_hss, dtype = int)
    pred_tos = checkTOs(predicted)
    pred_tos = np.asarray(pred_tos, dtype = int)
    pred_hss = checkHSs(predicted)
    pred_hss = np.asarray(pred_hss, dtype = int)

    tolerance = 300

## -- Check for the false transitions
    ref_tos, pred_tos = delete_false_transitions(ref_tos, pred_tos, 200)
    ref_hss, pred_hss = delete_false_transitions(ref_hss, pred_hss, 200)

    print("\n\n######### --- SUBJECT " + str(subj) + " --- #########\n\n")
    print("Reference baso: " + str(ref))
    print("Predicted baso: " + str(pred))

    count = 0

    print("\neference TOs:" + str(ref_tos))
    print("\nPredicted TOs:" + str(pred_tos))
    print("\nReference HSs:" + str(ref_hss))
    print("\nPredicted HSs:" + str(pred_hss))
    print("\nTolerance: " + str(tolerance))

    positives, tp, fp, fn, precision, recall, f1, diffs, ME, variance, SD = evaluate(ref_tos, pred_tos, tolerance)
    print("\nTOs:")
    print("Positives: " + str(positives))
    print("True Positives: " + str(tp))
    print("False Positives: " + str(fp))
    print("False Negatives: " + str(fn))
    print("Precision: " + str(precision))
    print("Recall: " + str(recall))
    print("F1: " + str(f1))
    print("Diffs: " + str(diffs))
    print("Mean Error: " + str(ME) + " samples")
    print("Variance: " + str(variance) + " samples")
    print("Standard Deviation: " + str(SD) + " samples")

    positives, tp, fp, fn, precision, recall, f1, diffs, ME, variance, SD = evaluate(ref_hss, pred_hss, tolerance)
    print("\nHSs:")
    print("Positives: " + str(positives))
    print("True Positives: " + str(tp))
    print("False Positives: " + str(fp))
    print("False Negatives: " + str(fn))
    print("Precision: " + str(precision))
    print("Recall: " + str(recall))
    print("F1: " + str(f1))
    print("Diffs: " + str(diffs))
    print("Mean Error: " + str(ME) + " samples")
    print("Variance: " + str(variance) + " samples")
    print("Standard Deviation: " + str(SD) + " samples")

sys.stdout = old_stdout
log_file.close()

In [57]:
import numpy as np
from numpy import genfromtxt as csv_load
import os
import pandas as pd

# def compute_threshold(v, phase):
in_path = '../subjects/min-max/windows_20/tr-False_sliding_1_c-False/folds_inter/Report_FF4/Fold_1/'
file = in_path + 's4_predicted_clean_350.csv'
v = csv_load(file, delimiter = ',')
phase = 1
phase_length = []
count = 0
print(v)
def checkHSs(v):
    hss = []
    count = 0
    for s in range(0,len(v)-1):
        count += 1
        if (v[s] == 1 and v[s+1] == 0):
            hss.append(count)
    return hss

def checkTOs(v):
    tos = []
    count = 0
    for s in range(0,len(v)-1):
        count += 1
        if (v[s] == 0 and v[s+1] == 1):
            tos.append(count)
    return tos

hss = checkHSs(v)
tos = checkTOs(v)
# hss.insert(0,0)



hss = np.asarray(hss, dtype = int)
tos = np.asarray(tos, dtype = int)

limit = min(len(hss),len(tos))
stances = np.zeros(limit, dtype = int)
swings = np.zeros(limit, dtype = int)


## -- Stance = hss - tos
for i in range(0, limit):
    stances[i] = tos[i] - hss[i]
    
mean_stance = np.mean(stances)
mean_stance = np.around(mean_stance, decimals = 2)
standard_dev_stance = np.std(stances)
standard_dev_stance = np.around(standard_dev_stance, decimals = 2)

print('Mean Stance +/- SD: ')
print(str(mean_stance) + ' +/- ' +  str(standard_dev_stance))

## -- Swing = tos - hss
for j in range(0, limit):
    swings[j] = abs(hss[j + 1] - tos[j])

mean_swing = np.mean(swings)
mean_swing = np.around(mean_swing, decimals = 2)
standard_dev_swing = np.std(swings)
standard_dev_swing = np.around(standard_dev_swing, decimals = 2)

print('Mean Stance +/- SD: ')
print(str(mean_swing) + ' +/- ' +  str(standard_dev_swing))

[1. 1. 1. ... 0. 0. 0.]
Mean Stance +/- SD: 
1194.76 +/- 149.18
Mean Stance +/- SD: 
959.07 +/- 119.64
55.471416035620265
